# Small Task Packing

This notebook demonstrates the workload behaviour for small task packing

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
# Generate plots inline
%pylab inline

import json
import os
import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [3]:
# Setup a target configuration
my_target_conf = {
    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : "/home/lisa/android-sdk",
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : '10.169.36.36',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
            "Device1" : 1, # iio:device0
        }
    },


    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    "host"        : '192.168.1.100',
    "username"    : 'root',
    "password"    : '',

    # Comment the following line to force rt-app calibration on your target


}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    #"tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
            
            
         ],
         "buffsize" : 10240
    },
    "results_dir" : "lisa_small_task_packing_2",
}

In [6]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

01:49:58  INFO    : Using base path: /media/build/workspace/lisa-github
01:49:58  INFO    : Loading custom (inline) target configuration
01:49:58  INFO    : Loading custom (inline) test configuration
01:49:58  WARNING : Wipe previous contents of the results folder:
01:49:58  WARNING :    /media/build/workspace/lisa-github/results/lisa_small_task_packing_2
01:49:58  INFO    : External tools using:
01:49:58  INFO    :    ANDROID_HOME: /media/build/workspace/android-sdk
01:49:58  INFO    :    CATAPULT_HOME: /media/build/workspace/lisa-github/tools/catapult
01:49:58  INFO    : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
01:49:58  INFO    : Connecting Android target [0123456789ABCDEF]
01:49:58  INFO    : Connection settings:
01:49:58  INFO    :    {'device': '0123456789ABCDEF'}
01:49:58  INFO    : Initializing target workdir:
01:49:58  INFO    :    /data/local/tmp/devlib-target
01:50:01  INFO    : Attempting to read energy model from target
01:50:05  INFO    : Topology:
01:50:05  I

# Workload configuration

In [7]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        'stp_0': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration
        ).get(),
        'stp_1': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration
            delay_s=0.01
        ).get(),
        'stp_2': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration    
            delay_s=0.02
        ).get(),
        'stp_3': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration    
            delay_s=0.035
        ).get(),
        'stp_4': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration
            delay_s=0.05
        ).get(),
        'stp_5': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration    
            delay_s=0.065
        ).get(),
        'stp_6': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration    
            delay_s=0.08
        ).get(),

    },    
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

01:50:10  INFO    : Setup new workload simple
01:50:10  INFO    : Workload duration defined by longest task
01:50:10  INFO    : Default policy: SCHED_OTHER
01:50:10  INFO    : ------------------------
01:50:10  INFO    : task [stp_0], sched: using default policy
01:50:10  INFO    :  | loops count: 1
01:50:10  INFO    : + phase_000001: duration 10.000000 [s] (100 loops)
01:50:10  INFO    : |  period   100000 [us], duty_cycle   2 %
01:50:10  INFO    : |  run_time   2000 [us], sleep_time  98000 [us]
01:50:10  INFO    : ------------------------
01:50:10  INFO    : task [stp_1], sched: using default policy
01:50:10  INFO    :  | start delay: 0.010000 [s]
01:50:10  INFO    :  | loops count: 1
01:50:10  INFO    : + phase_000001: duration 10.000000 [s] (100 loops)
01:50:10  INFO    : |  period   100000 [us], duty_cycle   2 %
01:50:10  INFO    : |  run_time   2000 [us], sleep_time  98000 [us]
01:50:10  INFO    : ------------------------
01:50:10  INFO    : task [stp_2], sched: using default pol

# Workload execution

In [8]:
# Set Sched governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('sched')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU max CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU max CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

01:50:14  INFO    : Target ABI: arm64, CPus: ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']
01:50:15  INFO    : Target current governor: sched
01:50:15  INFO    : Target big CPU max CPUfreq:2362000
		    Target big CPU current CPUfreq: 1421000
01:50:15  INFO    : Target LITTLE CPU max CPUfreq:1844000 
		    Target LITTLE CPU current CPUfreq: 1844000


In [9]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

01:50:17  INFO    : #### Setup FTrace
01:50:20  INFO    : #### Start energy sampling


01:50:21  INFO    : #### Start RTApp execution
01:50:21  INFO    : Workload execution START:
01:50:21  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
01:50:37  INFO    : #### Read energy consumption: /media/build/workspace/lisa-github/results/lisa_small_task_packing_2/energy.json
01:50:37  ERROR   : /usr/bin/iio-capture terminated for Device0 (iio:device0)
01:50:37  ERROR   : [ERROR: Unable to find host: Name or service not known
Unable to create IIO context
]
01:50:37  INFO    : #### Stop FTrace
01:50:37  INFO    : #### Save FTrace: /media/build/workspace/lisa-github/results/lisa_small_task_packing_2/trace.dat
01:50:39  INFO    : #### Save platform description: /media/build/workspace/lisa-github/results/lisa_small_task_packing_2/platform.json


# Collected results

In [10]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
#!ls -la {te.res_dir}

01:50:53  INFO    : Content of the output folder /media/build/workspace/lisa-github/results/lisa_small_task_packing_2


In [11]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

01:50:55  INFO    : Generated RTApp JSON file:


In [12]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_report.report_file)
print json.dumps(nrg_report.channels, indent=4, sort_keys=True)

01:50:57  INFO    : Energy: /media/build/workspace/lisa-github/results/lisa_small_task_packing_2/energy.json


{}


In [13]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

01:50:58  INFO    : Platform description: /media/build/workspace/lisa-github/results/lisa_small_task_packing_2/platform.json


# Trace inspection

In [14]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

# New Task Behavior

In [15]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['stp_0',   "stp_1", "stp_2",
                                             "stp_3", "stp_4", "stp_5", "stp_6"
                                            ])

/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.set_next_entity object at 0x7fa9edb83090> from cache, reading it from trace
  warnings.warn(warnstr)
/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.cpu_frequency_devlib object at 0x7fa9edb83190> from cache, reading it from trace
  warnings.warn(warnstr)
/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_move_numa object at 0x7fa9edb83290> from cache, reading it from trace
  warnings.warn(warnstr)
/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_wait_task object at 0x7fa9edb83390> from cache, reading it from trace
  warnings.warn(warnstr)
/media/build/workspace/lisa-github/libs/trappy/trappy/ftrace.py:205: UserWarning: TRAPpy: Couldn't read <trappy